# Matplotlib
Example of interactive plots

In [1]:
import pandas as pd
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import datetime

sns.set_theme()

In /home/tavant/.config/matplotlib/stylelib/presentation.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.


In [2]:
dataframe = pd.read_pickle("./data/THB_data.pkl").dropna()

mask = dataframe.index < datetime.datetime(2007,4,1)

small_dataframe = dataframe.iloc[mask]
small_dataframe.head()

,Bx,By,Bz,Np,Vx,Vy,Vz,Tp
2007-03-07 17:56:00,6.748950,-2.432548,14.107735,17.896076,3.322925,-100.716929,-276.617694,2.718395e+06
2007-03-07 17:57:00,6.822725,-3.480312,15.261338,77.188804,11.775027,26.851344,-81.346872,3.454449e+06
2007-03-07 17:58:00,5.256876,-2.683442,15.043352,58.404967,-9.468616,100.423579,-23.419666,3.028418e+06
2007-03-07 17:59:00,5.085039,-3.154718,16.192981,35.274029,-0.098382,135.313390,250.618605,2.746372e+06
2007-03-07 18:01:00,3.884535,-4.839686,12.379301,6.945407,-18.534470,-150.424068,-382.918429,5.036530e+06


# A simple plot

In [3]:
fig, ax = plt.subplots(1,1 )

ax.plot(small_dataframe.index, small_dataframe.Bx)

ax.set_xlabel("Date of the observation")
ax.set_ylabel("B field [unit]")
ax.set_title('Measurment of the Magnetif field')

plt.xticks(rotation=45)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# adding interaction to the plot

In matplotlib, interactions needs to :
- gate the line to change
- change the line data
- redraw

In [15]:
import ipywidgets as widgets

In [16]:
fig, ax = plt.subplots(1,1 )

line_Bfield, = ax.plot(small_dataframe.index, small_dataframe.Bx)

ax.set_xlabel("Date of the observation")
ax.set_ylabel("B field [unit]")
ax.set_title('Measurment of the Magnetif field')

plt.xticks(rotation=45)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
def update(change):
    """Update the line are redraw the figure
    The change argument depends of the widget
    The """

    line_Bfield.set_ydata(small_dataframe[change["new"]])
    ax.set_ylabel(change["new"] + " [unit]")
    
    # recompute the ax.dataLim
    ax.relim()
    # update ax.viewLim using the new dataLim
    ax.autoscale_view()

    fig.canvas.draw()
    

In [18]:
radiobuttons = widgets.RadioButtons(
    value='Bx', 
    options=list(small_dataframe.keys()), 
    description='Selection the data to plot'
)

In [19]:
radiobuttons.observe(update, 'value')

In [9]:
radiobuttons

RadioButtons(description='Selection the data to plot', options=('Bx', 'By', 'Bz', 'Np', 'Vx', 'Vy', 'Vz', 'Tp'…

Widgets can also be used as decoractors

In [20]:
# set up plot
fig, ax = plt.subplots(1,1 )

line_Bfield, = ax.plot(small_dataframe.index, small_dataframe.Bx)

ax.set_xlabel("Date of the observation")
ax.set_ylabel("B field [unit]")
ax.set_title('Measurment of the Magnetif field')

plt.xticks(rotation=45)
plt.tight_layout()

 
@widgets.interact(w=list(small_dataframe.keys()))
def update(w = "Bx"):
    """Remove old lines from plot and plot new one"""
    [l.remove() for l in ax.lines]
    ax.plot(small_dataframe.index, small_dataframe[w], color='C0')
    
    ax.set_ylabel(w + " [unit]")
    
    # recompute the ax.dataLim
    ax.relim()
    # update ax.viewLim using the new dataLim
    ax.autoscale_view()

    fig.canvas.draw()
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='w', options=('Bx', 'By', 'Bz', 'Np', 'Vx', 'Vy', 'Vz', 'Tp'), valu…

# interaction with the legend

The example shows how to have interaction between the figure legend and the plot.

It works, but is not that simple...

In [21]:
fig, ax = plt.subplots()
ax.set_title('Click on legend line to toggle line on/off')

line1, = ax.plot(small_dataframe.index, small_dataframe.Bx, lw=2,  label='Bx')
line2, = ax.plot(small_dataframe.index, small_dataframe.By, lw=2,  label='By')
leg = ax.legend(loc='upper left')
leg.get_frame().set_alpha(0.4)

# we will set up a dict mapping legend line to orig line, and enable
# picking on the legend line
lines = [line1, line2]
lined = dict()
for legline, origline in zip(leg.get_lines(), lines):
    legline.set_picker(5)  # 5 pts tolerance
    lined[legline] = origline


def onpick(event):
    # on the pick event, find the orig line corresponding to the
    # legend proxy line, and toggle the visibility
    legline = event.artist
    origline = lined[legline]
    vis = not origline.get_visible()
    origline.set_visible(vis)
    # Change the alpha on the line in the legend so we can see what lines
    # have been toggled
    if vis:
        legline.set_alpha(1.0)
    else:
        legline.set_alpha(0.2)
    fig.canvas.draw()

fig.canvas.mpl_connect('pick_event', onpick)

ax.set_xlabel("Date of the observation")
ax.set_ylabel("B field [unit]")

plt.xticks(rotation=45)
plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …